# [Skyline Problem](https://leetcode.com/problems/the-skyline-problem/)

- Give a list of `N` building
    + Each building represented by 3 values `[x_left, x_right, y]` (x = building coordinate, y = building height)

- Find the list of coordinate (x, y) to form the skyline

#### Constraints
- $0 \leq N \leq 10000$
- 0 $\leq$ x_left, x_right $\leq$ INT_MAX
- 0 $<$ y $\leq$ INT_MAX

#### Example

```
// Input
[2 9 10], [3 7 15], [5 12 12], [15 20 10], [19 24 8]

// Output
[2 10], [3 15], [7 12], [12 0], [15 10], [20 8], [24, 0]

// Explanation
```

| <img src="./assets/3.png" width="400"/> | <img src="./assets/4.jpg" width="400"/> |
|--------------------------------------|--------------------------------------|

# Solution

#### Segment-Intersection model O($N^2$)
- **Idea**
    + Use Segment-Intersection scan along x axis -> update max height
    + If max height changes -> update (x, y) to ans
    + Worst case:  O($N^2$) if all N buildings overlap
- Code

```Cpp
class Solution {
public:
    struct Building {
        int id;
        int x_left;
        int x_right;
        int y;
        Building() : id(0), x_left(0), x_right(0), y(0) {}
        Building(int id, int x_l, int x_r, int y) : id(id), x_left(x_l), x_right(x_r), y(y) {}
    };
    vector<vector<int>> getSkyline(vector<vector<int>>& buildings) {
        // Build Segment-Intersection model
        unordered_map<int, unordered_set<int>> start_ids;
        unordered_map<int, unordered_set<int>> end_ids;
        set<int> P;

        vector<Building> A;
        for(int i=0; i<buildings.size(); ++i) {
            vector<int> bd = buildings[i];
            int x_l = bd[0], x_r = bd[1], y = bd[2];

            A.push_back( Building(i, x_l, x_r, y) );
            start_ids[x_l].insert(i);
            end_ids[x_r].insert(i);
            P.insert(x_l);
            P.insert(x_r);
        }

        int cur_height = 0;
        unordered_set<int> cur_bd;
        vector<vector<int>> ans;
        for(const int &p:P) {
            // Update current building list
            for(const int &i:start_ids[p]) {
                cur_bd.insert(i);
            }
            for(const int &i:end_ids[p]) {
                cur_bd.erase(i);
            }

            // Update h: max height
            int h = 0;
            for(const int &i: cur_bd) {
                h = max(h, A[i].y);
            }

            // if h change -> Update ans
            if(cur_height != h) {
                cur_height = h;
                ans.push_back( {p, cur_height} );
            }
        }

        return ans;
    }
};
```

#### PQ - O(N logN)
- **Idea**
    + Use PQ to update current building list
        + `[y, x_right]: max y, min x_right`
        + at `x == p` If `x_right <= p` -> building out of current bd list, popped
    + Each building is pushed, popped only once -> Update max height reduced to O(N)

- Code

```Cpp
class Solution {
public:
    struct Building {
        int id;
        int x_left;
        int x_right;
        int y;
        Building() : id(0), x_left(0), x_right(0), y(0) {}
        Building(int id, int x_l, int x_r, int y) : id(id), x_left(x_l), x_right(x_r), y(y) {}
    };

    // max first - min second
    struct Compare {
        bool operator() (const pair<int,int> &a, const pair<int,int> &b) const {
            return (a.first < b.first) || (a.first == b.first && a.second > b.second);
        }
    };

    vector<vector<int>> getSkyline(vector<vector<int>>& buildings) {
        // Build Segment-Intersection model
        unordered_map<int, unordered_set<int>> start_ids;
        unordered_map<int, unordered_set<int>> end_ids;
        set<int> P;

        vector<Building> A;
        for(int i=0; i<buildings.size(); ++i) {
            vector<int> bd = buildings[i];
            int x_l = bd[0], x_r = bd[1], y = bd[2];

            A.push_back( Building(i, x_l, x_r, y) );
            start_ids[x_l].insert(i);
            end_ids[x_r].insert(i);
            P.insert(x_l);
            P.insert(x_r);
        }

        int cur_height = 0;
        priority_queue<pair<int,int>, vector<pair<int,int>>, Compare> pq;
            // {y, x_right}: max y - min x_right
        vector<vector<int>> ans;
        for(const int &p:P) {
            // Pop out out building have x_right <= p
            while(!pq.empty() && pq.top().second <= p) pq.pop();

            // Add building at x_left = p, update h: max height
            int h = pq.empty() ? 0 : pq.top().first;
            for(const int &i:start_ids[p]) {
                pq.push( {A[i].y, A[i].x_right} );
                h = max(h, A[i].y);
            }

            // if h change -> Update ans
            if(cur_height != h) {
                cur_height = h;
                ans.push_back( {p, cur_height} );
            }
        }
        return ans;
    }
};
```